In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 2771, done.
remote: Counting objects: 100% (889/889), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 2771 (delta 762), reused 804 (delta 692), pack-reused 1882
Receiving objects: 100% (2771/2771), 399.29 MiB | 47.74 MiB/s, done.
Resolving deltas: 100% (2214/2214), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.2 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
from optuna.distributions import FloatDistribution, IntDistribution, CategoricalDistribution
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


# Configurations

In [5]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [6]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [7]:
from os import system, getpid, walk
from psutil import Process
from colorama import Fore, Style, init
from IPython.display import display, HTML

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def GetMemUsage():   
    pid = getpid()
    py = Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return f"RAM memory GB usage = {memory_use :.4}"

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 0.2153


In [8]:
config = {
    'tune_parameters_item_knn': True,
    'tune_parameters_user_knn': True,
    'database_path': '/kaggle/working/better_tuning_knn.db',
    'copy_prev_tuning_db': True,
    'train_best_model': False,
    'copy_prev_best_params': True,
    'save_github': True
}

In [9]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/better_tuning_knn.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Data Import

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [13]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [14]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

# Hyperparameters tuning

In [15]:
import sqlite3

def print_all_rows(table_name, database_path):
    connection = sqlite3.connect(database_path)
    cursor = connection.cursor()

    try:
        cursor.execute(f"SELECT * FROM {table_name};")
        rows = cursor.fetchall()

        print(f"All rows from table '{table_name}':")
        for row in rows:
            print(row)

    except sqlite3.Error as e:
        print(f"Error reading data from table '{table_name}': {e}")

    finally:
        cursor.close()
        connection.close()

print_all_rows('studies', config['database_path'])

All rows from table 'studies':
(1, 'hyperparameters_tuning_knn_item_5folds')
(2, 'hyperparameters_tuning_knn_user_5folds')


## Item KNN Collaborative Filtering

Not considering adjusted cosine and pearson since they remove the mean of the interactions, but here there are all interactions equal to 1

In [16]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])


def objective_item_knn(trial):
    params = {
        'topK': trial.suggest_int('topK', 10, 1000),
        'shrink': trial.suggest_int('shrink', 0, 500),
        'similarity': trial.suggest_categorical('similarity', ["cosine", "asymmetric", "dice", "jaccard", "tanimoto", "tversky", "euclidean"]),
        'feature_weighting': trial.suggest_categorical('feature_weighting', ["BM25", "TF-IDF", "none"])
    }
    
    if params['similarity'] == "asymmetric":
        params["asymmetric_alpha"] = trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        params["normalize"] = True     

    elif params['similarity'] == "tversky":
        params["tversky_alpha"] = trial.suggest_float("tversky_alpha", 0, 2, log=False)
        params["tversky_beta"] = trial.suggest_float("tversky_beta", 0, 2, log=False)
        params["normalize"] = True 

    elif params['similarity'] == "euclidean":
        params["normalize_avg_row"] = trial.suggest_categorical("normalize_avg_row", [True, False])
        params["similarity_from_distance_mode"] = trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        params["normalize"] = trial.suggest_categorical("normalize", [True, False])
    
    recommender = ItemKNNCFRecommender(URM_train)
    recommender.fit(**params)
        
    result_df, _ = evaluator.evaluateRecommender(recommender)
    
    return result_df.loc[10, 'MAP']

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [17]:
%%time

if config['tune_parameters_item_knn']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_knn_item_no_cv', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_item_knn, n_trials=300)

[I 2023-12-09 10:44:45,803] A new study created in RDB with name: hyperparameters_tuning_knn_item_no_cv


ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4680.10 column/sec. Elapsed time 4.77 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 19.03 sec. Users per second: 549


[I 2023-12-09 10:45:10,505] Trial 0 finished with value: 0.03742553831019832 and parameters: {'topK': 937, 'shrink': 93, 'similarity': 'dice', 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.03742553831019832.


ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 489.74 column/sec. Elapsed time 45.63 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 8.23 sec. Users per second: 1270


[I 2023-12-09 10:46:04,911] Trial 1 finished with value: 0.019689644284135395 and parameters: {'topK': 516, 'shrink': 261, 'similarity': 'euclidean', 'feature_weighting': 'TF-IDF', 'normalize_avg_row': True, 'similarity_from_distance_mode': 'lin', 'normalize': False}. Best is trial 0 with value: 0.03742553831019832.


...

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4536.80 column/sec. Elapsed time 4.93 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 11.53 sec. Users per second: 907


[I 2023-12-09 12:12:42,143] Trial 298 finished with value: 0.03311541334207781 and parameters: {'topK': 62, 'shrink': 488, 'similarity': 'tversky', 'feature_weighting': 'none', 'tversky_alpha': 0.08352542974766058, 'tversky_beta': 1.8386624360056547}. Best is trial 276 with value: 0.04687268493217283.


ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4598.01 column/sec. Elapsed time 4.86 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 9.47 sec. Users per second: 1104


[I 2023-12-09 12:12:56,897] Trial 299 finished with value: 0.04685008455690415 and parameters: {'topK': 10, 'shrink': 468, 'similarity': 'tversky', 'feature_weighting': 'BM25', 'tversky_alpha': 0.22296625283643304, 'tversky_beta': 1.9749601437127569}. Best is trial 276 with value: 0.04687268493217283.


CPU times: user 1h 20min 1s, sys: 7min 46s, total: 1h 27min 48s
Wall time: 1h 28min 11s


In [18]:
if config['tune_parameters_item_knn']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [19]:
if config['tune_parameters_item_knn']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [20]:
if config['tune_parameters_item_knn']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [21]:
if config['tune_parameters_item_knn']:
    with open('/kaggle/working/best_params_knn_item_no_cv.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_knn_item_no_cv.json', 
            'tuning_results/best_params_knn_item_no_cv.json', 
            'Item KNN tuning results without cross validation (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item_no_cv.json', 
        '/kaggle/working/best_params_knn_item_no_cv.json'
    )

## User KNN Collaborative Filtering

In [22]:
def objective_user_knn(trial):
    params = {
        'topK': trial.suggest_int('topK', 10, 1000),
        'shrink': trial.suggest_int('shrink', 0, 500),
        'similarity': trial.suggest_categorical('similarity', ["cosine", "asymmetric", "dice", "jaccard", "tanimoto", "tversky", "euclidean"]),
        'feature_weighting': trial.suggest_categorical('feature_weighting', ["BM25", "TF-IDF", "none"])
    }
    
    if params['similarity'] == "asymmetric":
        params["asymmetric_alpha"] = trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        params["normalize"] = True     

    elif params['similarity'] == "tversky":
        params["tversky_alpha"] = trial.suggest_float("tversky_alpha", 0, 2, log=False)
        params["tversky_beta"] = trial.suggest_float("tversky_beta", 0, 2, log=False)
        params["normalize"] = True 

    elif params['similarity'] == "euclidean":
        params["normalize_avg_row"] = trial.suggest_categorical("normalize_avg_row", [True, False])
        params["similarity_from_distance_mode"] = trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        params["normalize"] = trial.suggest_categorical("normalize", [True, False])
    
    recommender = UserKNNCFRecommender(URM_train)
    recommender.fit(**params)
        
    result_df, _ = evaluator.evaluateRecommender(recommender)
    
    return result_df.loc[10, 'MAP']

In [23]:
%%time

if config['tune_parameters_user_knn']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_knn_user_no_cv',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_user_knn, n_trials=100)

[I 2023-12-09 12:13:11,178] A new study created in RDB with name: hyperparameters_tuning_knn_user_no_cv


UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 5269.99 column/sec. Elapsed time 2.47 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 10.39 sec. Users per second: 1007


[I 2023-12-09 12:13:24,453] Trial 0 finished with value: 0.030309619448391733 and parameters: {'topK': 88, 'shrink': 50, 'similarity': 'jaccard', 'feature_weighting': 'TF-IDF'}. Best is trial 0 with value: 0.030309619448391733.


UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4725.96 column/sec. Elapsed time 2.76 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 10.73 sec. Users per second: 975


[I 2023-12-09 12:13:38,187] Trial 1 finished with value: 0.029586665512928156 and parameters: {'topK': 184, 'shrink': 132, 'similarity': 'cosine', 'feature_weighting': 'none'}. Best is trial 0 with value: 0.030309619448391733.


...

UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4524.20 column/sec. Elapsed time 2.88 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 11.94 sec. Users per second: 875


[I 2023-12-09 12:38:44,234] Trial 98 finished with value: 0.0314316569206109 and parameters: {'topK': 404, 'shrink': 260, 'similarity': 'cosine', 'feature_weighting': 'none'}. Best is trial 92 with value: 0.03477967385628101.


UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4978.46 column/sec. Elapsed time 2.62 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 12.27 sec. Users per second: 853


[I 2023-12-09 12:38:59,596] Trial 99 finished with value: 0.03195169705857381 and parameters: {'topK': 437, 'shrink': 200, 'similarity': 'tanimoto', 'feature_weighting': 'TF-IDF'}. Best is trial 92 with value: 0.03477967385628101.


CPU times: user 23min 5s, sys: 2min 36s, total: 25min 41s
Wall time: 25min 48s


In [24]:
if config['tune_parameters_user_knn']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [25]:
if config['tune_parameters_user_knn']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [26]:
if config['tune_parameters_user_knn']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [27]:
if config['tune_parameters_user_knn']:
    with open('/kaggle/working/best_params_knn_user_no_cv.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_knn_user_no_cv.json', 
            'tuning_results/best_params_knn_user_no_cv.json', 
            'User KNN tuning results without cross validation (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user_no_cv.json', 
        '/kaggle/working/best_params_knn_user_no_cv.json'
    )

In [28]:
if config['save_github'] and (config['tune_parameters_user_knn'] or config['tune_parameters_item_knn']):
    upload_file(
        config['database_path'], 
        'tuning_results/better_tuning_knn.db', 
        'Tuning db updated results (from kaggle notebook)'
    )

# Evaluating best models on the validation set

In [29]:
# with open('/kaggle/working/best_params_knn_user_no_cv.json', 'r') as params_file:
#     params_user = json.load(params_file)
    
# with open('/kaggle/working/best_params_knn_item_no_cv.json', 'r') as params_file:
#     params_item = json.load(params_file)

In [30]:
# evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

# recommender = ItemKNNCFRecommender(URM_train)
# recommender.fit(**params_item)

# result_df, _ = evaluator.evaluateRecommender(recommender)
# print(f'Item KNN')
# display(result_df)

# recommender = UserKNNCFRecommender(URM_train)
# recommender.fit(**params_user)

# result_df, _ = evaluator.evaluateRecommender(recommender)
# print(f'User KNN')
# display(result_df)